## Step 1: Setup and Imports

In [2]:
# Import packages
import os
import zipfile
import subprocess
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm


from Bio import SeqIO
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Errors ignore
import warnings
warnings.filterwarnings('ignore')

## Step 2: Extract Zip Files

In [ ]:
# Define paths to zip files and output files

zip_files = {'TrainFiles.zip': './',
             'TestFiles.zip': './'}

for zip_path, output_path in tqdm(zip_files.items(), desc="Extracting Files"):
    os.makedirs(output_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_path)
        print(f"Extracted {zip_path} to ./ {output_path}/")


Extracting Files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting Files:  50%|█████     | 1/2 [07:53<07:53, 473.81s/it]

Extracted TrainFiles.zip to ./ .//


Extracting Files: 100%|██████████| 2/2 [09:54<00:00, 297.31s/it]

Extracted TestFiles.zip to ./ .//


## Step 3: Decoding of Files
### Decode a single file first and then all .mgb files to fastq files

In [ ]:
# Try one mgb file from the TrainFiles

mgb_filename = 'ID_ABOEMW.mgb'
mgb_filename_without_ext = mgb_filename[:-4]
train_dir = os.path.join(os.getcwd(), "TrainFiles")
mgb_file_path = os.path.join(train_dir, mgb_filename)

fastq_dir = os.path.join(os.getcwd(), "FastqTrain")
os.makedirs(fastq_dir, exist_ok=True)  # make sure folder exists
#Output fastq file
output_fastq = os.path.join(fastq_dir, f"{mgb_filename_without_ext}.fastq")

container_dir = "/data"

def inspect_mgb_structure(host_dir=".", container_dir="/work", mgb_filename=mgb_filename):
    command = [
    "docker", "run", "--rm",
    "-v", f"{train_dir}:/input",
    "-v", f"{fastq_dir}:/output",
    "muefab/genie:latest", "run", 
    "-f",
    "-i", f"/input/{mgb_filename}",
    "-o", f"/output/{mgb_filename_without_ext}.fastq"
    ]

    print("Running Genie")
    print(" ".join(command))

    result = subprocess.run(command, capture_output=True, text=True)

    if result.stdout:
        print("\n STDOUT:\n", result.stdout)
    
    if result.stderr:
        print("\n STDERR:\n", result.stderr)

inspect_mgb_structure()



In [ ]:

fastq_path = os.path.join(os.getcwd(), fastq_dir, f"{mgb_filename_without_ext}.fastq")

# Check if the file exists
if not os.path.exists(fastq_path):
    print("Fastq file not found")
else:
    total_reads = 0
    read_lengths = []
    quality_scores = []

    for record in SeqIO.parse(fastq_path, "fastq"):
        total_reads += 1
        read_lengths.append(len(record.seq))
        quality_scores.extend(record.letter_annotations["phred_quality"])

    print(f"🔍 Total reads: {total_reads}")
    print(f"📏 Avg read length: {sum(read_lengths)/len(read_lengths):.1f} bp")
    print(f"🎯 Avg quality score: {sum(quality_scores)/len(quality_scores):.1f}")



In [ ]:
# View first 4 reads

for i, record in enumerate(SeqIO.parse(fastq_path, "fastq")):
    print(f"ID: {record.id}")
    print(f"SEQ {record.seq[:60]}")
    print(f"Quality: {record.letter_annotations['phred_quality'][:20]}...\n")

    if i >=3:
        break


In [ ]:

# Function to decode all .mgb files to fastq files
notebook_dir = os.getcwd()

def decode_all_mgb_files(source_folder, destination_folder):
    host_dir = os.path.join(notebook_dir, source_folder)
    fastq_dir = os.path.join(notebook_dir, destination_folder)
    os.makedirs(fastq_dir, exist_ok=True)

    if not os.path.isdir(host_dir):
        print(f"Error: Source folder '{host_dir}' does not exist or is not a directory.")
        return # Exit the function if source folder is invalid

    for mgb_filename in tqdm(os.listdir(host_dir), desc="Decoding files..."):
        name, ext = os.path.splitext(mgb_filename)

        if ext.lower() != ".mgb":
            continue

        #mgb_filename_without_ext = os.path.splitext(mgb_filename)[0]

        output_fastq_path = os.path.join(fastq_dir, f"{name}.fastq")

        if os.path.exists(output_fastq_path):
            print(f"Skipping {mgb_filename} already decoded")
            continue

        command = [
            "docker", "run", "--rm",
            "-v", f"{host_dir}:/input",
            "-v", f"{fastq_dir}:/output",
            "muefab/genie:latest", "run", 
            "-f",
            "-i", f"/input/{mgb_filename}",
            "-o", f"/output/{name}.fastq"
        ]

        print("Running command:", " ".join(command))
        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"Failed to decode {mgb_filename}")
            print(f"STDOUT {result.stdout}")
            print(f"STDERR {result.stderr}")

In [ ]:
decode_all_mgb_files("TrainFiles", "FastqTrain")

In [ ]:
decode_all_mgb_files("TestFiles", "FastqTest")

In [ ]:
#full_train_df = train_df.merge(subjects_df, how='left', on='SubjectID')
#full_train_df.head(50)

In [ ]:
fastq_dir = os.getcwd() + "/FastqTrain"
all_stats = []

for fname in tqdm(os.listdir(fastq_dir), desc="Processing files..."):
    if not fname.endswith(".fastq"):
        continue
    file_path = os.path.join(fastq_dir, fname)
    read_lengths = []
    gc_counts = []
    nt_counts = Counter()

    for record in SeqIO.parse(file_path, "fastq"):
        seq = str(record.seq)
        read_lengths.append(len(seq))
        gc_counts.append(seq.count("G") + seq.count("C"))
        nt_counts.update(seq)

    if read_lengths:
        stats = {
            "file": fname,
            "num_reads": len(read_lengths),
            "avg_read_length": sum(read_lengths) / len(read_lengths),
            "avg_gc_content": sum(gc_counts) / sum(read_lengths),
            "A": nt_counts["A"],
            "T": nt_counts["T"],
            "G": nt_counts["G"],
            "C": nt_counts["C"]

        }

        all_stats.append(stats)

    # Convert to Dataframe
df = pd.DataFrame(all_stats)


Processing files...: 100%|██████████| 2901/2901 [4:18:45<00:00,  5.35s/it]    


: 

In [1]:
#df["filename"] = df["file"].str.replace(".fastq", "", regex=False)
df.head()

NameError: name 'df' is not defined